# Kickstarter Startup Analysis

1. How should we design our Kickstarter campaign in order to make our startup funding phase as succesful as possible?
    - **Categorie** (Assumption Tech) - Succesful campaigns
        -> ratio succesful/unsuccesful/live definieren
        -> one-sample 1 tegenover het totaal
        -> two sample 1 tegenover de rest, of Tech vs Design
    - **US vs NL** (Assumption: Silicon Valley vs. the Netherlands)
        -> Average pledged amount in NL hoger OF lager dan in US (1 of 2 sample T test)
    - **Number of backers** (Assumption: popularity of campaign defines popularity of startup)
        -> Ga je voor weinig backers met veel of veel backers met weinig -> verschil in succesful (Hoeveel backers per in je goal)
    -> Zoja/Zonee -> define confidence intervals voor backers voor ons "start-up" plan
    
    - **Pledge amount asked** - Pledged (goal - converted)
    
2. Which pledge amount will lead to the highest possible funding?

3. How could the location of the kickstarter campaign, or the sub-category, influence the success rate of our kickstarter campaign?

**To do**  
~~Elske: Fix git issues  ~~  
Rianne: #Drop duplicates (per project_id, check categorieen - 1, 2 en 3 -- zijn hetzelfde, dus sort_values by pledged amount, keepfirst)  
Rianne: #Currency -> use usd_pledged -> convert_amount can be dropped, fx_rate also, currency also   
Rianne: #Take out live  
~~Elske:  #Canceled and unsuccesful -> Failed~~  
~~Elske:  #Unfold Dict "location" unfold: 'name', 'expanded_country'~~  
~~Elske:  #Create column with average pledge per backer  ~~  
Both: Check for missing values and how to solve them + add code for hypothesis testing

In [10]:
#Importing libraries
import pandas as pd
import numpy as np
import json
from datetime import datetime

Download the file <a href="https://drive.google.com/drive/folders/1hIBnfoXrXNW2vF_ka8R8Y0mltCmBjqPZ" target="_blank">here</a>.

In [5]:
#Importing the data using a relative path
#Mac
df = pd.read_csv('../Kickstarter_analysis/sep2020.csv', converters={'category': eval})
df.head()

#Windows
df = pd.read_csv('data\\sep2020.csv')
df.head()

,backers_count,blurb,country,created_at,deadline,goal,id,launched_at,name,pledged,spotlight,state,state_changed_at,usd_pledged,category_id,category_name,category_parent_id,category_parent_name,project_id
0,61,Support great art! Join us as we re-stage the ...,US,1579292017,1583025192,3000.0,1568973024,1580433192,Revisionist History 2,4618.0,True,successful,1583025192,4618.0,254,Performances,6.0,Dance,3914215
1,52,JinBucha is a new kind of Brewery in North Par...,US,1446051515,1450118057,20000.0,1782378629,1447526057,JinBucha: A Modern Kombucha tasting room in No...,3461.0,False,failed,1450118057,3461.0,307,Drinks,10.0,Food,2198010
2,0,"""Taste The Scent of Tea"" Rose and Blanc Tea Ro...",US,1518159717,1520233260,15000.0,2076142479,1518208887,Tea Room Cafe - Daily & Herb Tea,0.0,False,failed,1520233260,0.0,307,Drinks,10.0,Food,3304826
3,2,Bringing Philly cheesesteaks (and other delici...,US,1454217596,1459885844,2000.0,1416797221,1454705444,HELP TWO BROTHERS OPEN A FOOD TRUCK!,25.0,False,failed,1459885844,25.0,311,Food Trucks,10.0,Food,2349147
4,150,a frog plushie keychain and frog butt pin for ...,US,1571342290,1574445600,800.0,1708708142,1573236000,Son the Frog Plushie Keychain + JUICY Pin,3275.0,True,successful,1574445600,3275.0,28,Product Design,7.0,Design,3834865


**Step 1:** Data Cleaning and Wrangling
- Transforming unix time stamps to datetime

In [25]:
# Dates transformed [['created_at'], ['deadline'], ['launched_at'], ['state_changed_at']]
df[['created_at','deadline', 'launched_at', 'state_changed_at']].apply(pd.to_datetime, unit='s')

,created_at,deadline,launched_at,state_changed_at
0,2020-01-17 20:13:37,2020-03-01 01:13:12,2020-01-31 01:13:12,2020-03-01 01:13:12
1,2015-10-28 16:58:35,2015-12-14 18:34:17,2015-11-14 18:34:17,2015-12-14 18:34:17
2,2018-02-09 07:01:57,2018-03-05 07:01:00,2018-02-09 20:41:27,2018-03-05 07:01:00
3,2016-01-31 05:19:56,2016-04-05 19:50:44,2016-02-05 20:50:44,2016-04-05 19:50:44
4,2019-10-17 19:58:10,2019-11-22 18:00:00,2019-11-08 18:00:00,2019-11-22 18:00:00
...,...,...,...,...
219683,2019-02-21 18:33:41,2019-03-29 21:00:00,2019-02-27 15:12:10,2019-03-29 21:00:02
219684,2013-03-05 20:13:03,2013-08-10 07:45:40,2013-07-05 07:45:40,2013-08-10 07:45:40
219685,2016-06-21 00:35:01,2016-08-13 21:49:58,2016-07-14 21:49:58,2016-08-13 21:49:58
219686,2015-11-28 19:14:15,2016-01-09 19:19:33,2015-12-10 19:19:33,2016-01-09 19:19:34


- Remove live kickstarter campaings.

In [30]:
# Drop records of unfinished project (category live)
df = df.loc[df['state'] != 'live']
len(df)

188614

- Drop duplicated projects

In [38]:
# Drop duplicates
df_duplicated = df.loc[df['project_id'].duplicated() == True]
df_duplicated

,backers_count,blurb,country,created_at,deadline,goal,id,launched_at,name,pledged,...,state,state_changed_at,usd_pledged,category_id,category_name,category_parent_id,category_parent_name,project_id,state_cat,average_pledge
51364,107,A romantic comedy in graphic novel form. The t...,GB,1581595491,1586476803,500.0,679682143,1583884803,You're Stuck With Me Now,2335.00,...,successful,1586476803,3054.004221,252,Graphic Novels,3.0,Comics,3933474,successful,28.542096
106725,30,"No joints, No metal frame.\nCustom engraving o...",NZ,1569979404,1574562838,2000.0,1074657626,1571967238,PuroBerg | The World's Best Craftmanship Woode...,3104.42,...,successful,1574562838,1984.034015,28,Product Design,7.0,Design,3824616,successful,66.134467
121474,1722,💀 A survival card game set in the Australian O...,AU,1570947350,1595379558,3000.0,255790389,1593565158,🐨 Lost In Straya - A fast & fun survival card ...,48797.32,...,successful,1595379558,33580.558433,34,Tabletop Games,12.0,Games,3831662,successful,19.500905
127382,38,"Re-seeding life, one T-shirt at a time.",JP,1587347962,1589781600,300000.0,738793616,1588297480,SURU - Japanese Manhole Cover Bamboo T-shirts ...,303459.00,...,successful,1589781600,2848.111410,263,Apparel,9.0,Fashion,3978126,successful,74.950300
129066,44,★★★★ 'Hauntingly beautiful!' (Ask The Ushers)\...,GB,1521237420,1557352800,4500.0,1856292915,1552178909,DROUGHT in HOLLYWOOD,4500.00,...,successful,1557352800,5856.324840,281,Experimental,17.0,Theater,3335535,successful,133.098292
132606,2075,《亂世備忘》陳梓桓導演．《地厚天高》《亂世備忘》任硯聰監製．《十年》蔡廉明聯合監製,HK,1587631956,1593964800,800000.0,1740554502,1591069408,《憂鬱之島》Blue Island | 香港抗爭紀錄片 A Film on Hong Kon...,1651133.53,...,successful,1593964800,212982.157712,30,Documentary,11.0,Film & Video,3980663,successful,102.642004
135591,79,"Our own developed time-telling mechanism, supe...",HK,1575615163,1580086906,40000.0,830442676,1577494906,SPACESHIP - the unique time-telling mechanical...,644640.00,...,successful,1580086907,82762.170864,28,Product Design,7.0,Design,3866545,successful,1047.622416
167123,58,A play adapted from award-winning graphic nove...,GB,1532019555,1534114800,1650.0,1851866112,1532046851,The Art of Charlie Chan Hock Chye (A Play),1827.00,...,successful,1534114800,2395.261402,285,Plays,17.0,Theater,3429084,successful,41.297610
178218,899,Portraits of the Hong Kong Protesters and Thei...,HK,1588237146,1591891200,150000.0,1749269487,1588729246,Wounds of Hong Kong 港傷,209152.00,...,successful,1591891200,26970.476677,280,Photobooks,15.0,Photography,3985359,successful,30.000530


In [29]:
df.loc[df['project_id'] == 3978126].transpose()

,113324,127382
backers_count,38,38
blurb,"Re-seeding life, one T-shirt at a time.","Re-seeding life, one T-shirt at a time."
country,JP,JP
created_at,1587347962,1587347962
deadline,1589781600,1589781600
goal,300000,300000
id,738793616,738793616
launched_at,1588297480,1588297480
name,SURU - Japanese Manhole Cover Bamboo T-shirts ...,SURU - Japanese Manhole Cover Bamboo T-shirts ...
pledged,303459,303459


- Classify cancelled and unsuccesful as both "unsuccesful"  
Since we are looking to launch a succesful campaign for our start-up, 
we categorize the data in two categories: "Succesful" and "Failed" (compromising canceled and failed campaings).

In [14]:
#Value_counts shows there are both canceled and failed campaigns. Categorize into successfull and unsuccessful
df['state_cat'] = np.where(df['state']=='successful', 'successful', 'unsuccessful')
df['state_cat'].value_counts()

successful      128156
unsuccessful     85275
Name: state_cat, dtype: int64

- Create variable 'average_pledge' per backer

In [16]:
df['average_pledge'] = (df['usd_pledged']/df['backers_count'])
df['average_pledge'].head()

0    75.704918
1    66.557692
2          NaN
3    12.500000
4    21.833333
Name: average_pledge, dtype: float64

- Check for missing values

In [18]:
df.isna().sum()

backers_count                   0
blurb                           8
converted_pledged_amount        0
country                         0
created_at                      0
currency                        0
deadline                        0
fx_rate                         0
goal                            0
id                              0
launched_at                     0
location                      215
name                            0
pledged                         0
spotlight                       0
state                           0
state_changed_at                0
usd_pledged                     0
category_id                     0
category_name                   0
category_parent_id           8300
category_parent_name         8300
project_id                      0
average_pledge              16133
dtype: int64

# Data Exploration

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
#Grouping the df by main category, sorted by sum of converted_pledged amount.
top_pledged = df.loc[:,df['state'] == 'succesful'].groupby(['category_parent_name']).sum().reset_index().sort_values(by='converted_pledged_amount', ascending=False)
top_backers = df.groupby(['category_parent_name']).sum().reset_index().sort_values(by='backers_count', ascending=False)
#Selecting top five from top_pledged and top_backers
top_five_pledged = top_pledged.head(5)
top_five_backers = top_backers.head(5)
pal_pled = sns.color_palette("Greens_d", len(top_five_pledged))
pal_back = sns.color_palette("Reds_d", len(top_five_backers))
rank_pled = top_five_pledged["converted_pledged_amount"].argsort().argsort()
rank_back = top_five_backers["backers_count"].argsort().argsort()
#Plotting the top five
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 4))
sns.barplot(ax =axes[0], x='converted_pledged_amount',y='category_parent_name', data=top_five_pledged, linewidth=0.7, palette=np.array(pal_pled[::+1])[rank_pled])
axes[0].set_title('Converted Amount Pledged')
sns.barplot(ax=axes[1], x='backers_count',y='category_parent_name', data=top_five_backers, linewidth=0.7, palette=np.array(pal_back[::+1])[rank_back])
axes[1].set_title('Number of Backers');

In [ ]:
plt.figure(figsize=(15, 10))
sns.boxplot(x='category_parent_name', y='converted_pledged_amount', hue='state', data=df, orient='v', showfliers=False);
plt.title('Converted Pledged Amount per Category Parent', fontsize=20);
plt.xlabel('Category Parent', fontsize=18)
plt.ylabel('Converted Pledged Amount)', fontsize=18)
plt.xticks(fontsize=10)
plt.yticks(fontsize=15);

#Can we use "proportion of the population" - om significant verschil tussen succesful (tech en de rest)


In [ ]:
df.groupby(['category_name']).sum().sort_values(by='converted_pledged_amount', ascending=False)

In [ ]:
#corr
corr = df['converted_pledged_amount'].corr(df['backers_count'])
#set figure size
plt.figure(figsize=(15, 10))
#plot the data
sns.regplot(data=df,x=df['converted_pledged_amount'],y=df['backers_count'], ci=None)
#format the plot
text = 'r=%s' % (corr)
plt.text(1, 19, text, fontsize=14)
plt.title('', fontsize=20);
plt.xlabel('', fontsize=18)
plt.ylabel('', fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18);

# Statistical Analysis

In [25]:
#Confidence Intervals:
# To define between pledge amount x and x we should ask
# To define between number of backers x and x we should aim for

# One-sample
# 1 category opposed to population mean

# Two-sample
#2 categories (tech vs. food) and Netherlands/Europe vs. US

- Categorie (Assumption Tech) 
        - Succesful campaigns
        -> ratio succesful/unsuccesful/live definieren
        -> one-sample 1 tegenover het totaal
        -> two sample 1 tegenover de rest, of Tech vs Design 
Elske

- US vs NL (Assumption: Silicon Valley vs. the Netherlands)
    - Usd pledged in NL hoger OF lager dan in US (1 of 2 sample T test)  
    -> Dict "location" needs to be unfolded ('name', 'expanded_country')  
    -> Test binnen alle categorien eerst, eventueel later alleen in hierboven gekozen categorie  
Rianne

- Number of backers (Assumption: popularity of campaign defines popularity of startup)
    - Ga je voor weinig backers met veel of veel backers met weinig -> verschil in succesful  
        -> Define average pledge per backer (see to do)  
        -> Succesful versus unsuccesful (2-sample: avg. per backer succesful versus avg. per backer unsuccesful)  
            We verwachten dat dit verschilt - gemiddelde bij succesvol hoger
Rianne  
            
- Zoja/Zonee -> define confidence intervals voor backers voor ons "start-up" plan (binnen onze categorie?)  

Elske

Bonus:
In specific categories: Pledge amount asked - Pledged (goal - converted)  
COVID impact 2020 versus 2018/2019 (less pledges than previous years - and in which categories?)  
Other differences in years   
    